In [1]:
%load_ext autoreload
%autoreload 3

In [126]:
import pytest
import numpy as np
from minitorch.tensor.tensor import Tensor
from minitorch.autograd.autograd import (AddBackward,
                            MulBackward,
                            MatMulBackward,
                            DivBackward,
                            SubBackward,
                            TransposeBackward)

In [112]:
def tensor(data, requires_grad=True):
    """create a tensor using the Tensor class

    Args:
        data (list,ndarray): data to use to create the tensor
        requires_grad (bool, optional): _description_. Defaults to True.

    Returns:
        Tensor: N-dimension tensor array storing the data
    """
    return Tensor(np.array(data, dtype= np.float32), requires_grad=requires_grad)

In [79]:
def test_add_backward_basic():
    a = tensor([1.0, 2.0])
    b = tensor([3.0, 4.0])
    grad_out = tensor([1.0, 1.0], requires_grad=False)

    fn = AddBackward(a, b)
    grad_a, grad_b = fn(grad_out)
    print(grad_a, grad_b)

    np.testing.assert_allclose(grad_a.data, grad_out.data)
    np.testing.assert_allclose(grad_b.data, grad_out.data)


def test_add_backward_requires_grad_respected():
    a = tensor([1.0, 2.0], requires_grad=False)
    b = tensor([3.0, 4.0], requires_grad=True)
    grad_out = tensor([1.0, 1.0], requires_grad=False)

    fn = AddBackward(a, b)
    grad_a, grad_b = fn(grad_out)
    print(grad_a, grad_b)
    
    test_grad = Tensor(np.zeros_like(a.data))
    
    np.testing.assert_allclose(grad_a.data, test_grad.data)
    np.testing.assert_allclose(grad_b.data, grad_out.data)


def test_add_backward_invalid_input():
    with pytest.raises(AssertionError):
        fn = AddBackward(1, 2)
        fn(tensor(1.0))
        
test_add_backward_basic()
test_add_backward_requires_grad_respected()
test_add_backward_invalid_input()

Tensor(data=[1. 1.]) Tensor(data=[1. 1.])
Tensor(data=[0. 0.]) Tensor(data=[1. 1.])


In [80]:
def test_sub_backward_basic():
    a = tensor([1.0, 2.0])
    b = tensor([3.0, 4.0])
    grad_out = tensor([1.0, 1.0], requires_grad=False)

    fn = SubBackward(a, b)
    grad_a, grad_b = fn(grad_out)
    print(grad_a, grad_b)
    

    np.testing.assert_allclose(grad_a.data, grad_out.data)
    np.testing.assert_allclose(grad_b.data, -grad_out.data)


def test_sub_backward_requires_grad_respected():
    a = tensor([1.0, 2.0], requires_grad=False)
    b = tensor([3.0, 4.0], requires_grad=False)
    grad_out = tensor([1.0, 1.0], requires_grad=False)

    fn = AddBackward(a, b)
    grad_a, grad_b = fn(grad_out)
    test_grad = Tensor(np.zeros_like(a.data))
    print(grad_a, grad_b)
    np.testing.assert_allclose(grad_a.data, test_grad.data)
    np.testing.assert_allclose(grad_b.data, test_grad.data)


def test_add_backward_invalid_input():
    with pytest.raises(AssertionError):
        fn = SubBackward(1, 2)
        fn(tensor(1.0))
   
test_sub_backward_basic()
test_sub_backward_requires_grad_respected()
test_add_backward_invalid_input()

Tensor(data=[1. 1.]) Tensor(data=[-1. -1.])
Tensor(data=[0. 0.]) Tensor(data=[0. 0.])


In [85]:
def test_div_backward_basic():
    a = tensor([6.0, 8.0])
    b = tensor([2.0, 4.0])
    grad_out = tensor([20.0, 16.0], requires_grad=False)

    fn = DivBackward(a, b)
    grad_a, grad_b = fn(grad_out)
    print(grad_a, grad_b)
    
    np.testing.assert_allclose(
        grad_a.data,
        grad_out.data / b.data
    )

    np.testing.assert_allclose(
        grad_b.data,
        -grad_out.data * a.data / (b.data ** 2)
    )


def test_div_backward_requires_grad_respected():
    a = tensor([6.0, 8.0], requires_grad=False)
    b = tensor([2.0, 4.0], requires_grad=True)
    grad_out = tensor([1.0, 1.0], requires_grad=False)

    fn = DivBackward(a, b)
    grad_a, grad_b = fn(grad_out)
    test_grad = Tensor(np.zeros_like(grad_out.data))
    print(grad_a, grad_b)

    np.testing.assert_allclose(grad_a.data, test_grad.data)
    np.testing.assert_allclose(
        grad_b.data,
        -grad_out.data * a.data / (b.data ** 2)
    )
    
test_div_backward_basic()
test_div_backward_requires_grad_respected()

Tensor(data=[10.  4.]) Tensor(data=[-30.  -8.])
Tensor(data=[0. 0.]) Tensor(data=[-1.5 -0.5])


In [135]:
def test_transpose_backward_basic():
    a = tensor([[6.0, 8.0], [2.0, 4.0]])
    # b = tensor([2.0, 4.0])
    grad_out = tensor([[20.0, 16.0], [18.0, 12.0]], requires_grad=False)

    fn = TransposeBackward(a,0,1)
    grad_a = fn(grad_out)
    test_grad = grad_out.transpose(0,1)
    
    print(grad_a)
    print(test_grad)
    np.testing.assert_allclose(
        grad_a[0].data,
        test_grad.data
    )



test_transpose_backward_basic()

(Tensor(data=[[20. 18.]
 [16. 12.]], shape=(2, 2), grad_info= None),)
Tensor(data=[[20. 18.]
 [16. 12.]])


In [142]:
Tensor().grad

TypeError: Tensor.__init__() missing 1 required positional argument: 'data'

In [ ]:
def enable_autograd(quiet=False):
    #* GUARD: prevent double patching
    if hasattr(Tensor, '_autograd_enabled'):
        #* silently return the tensor if already enable - no need to warn 
        return 
    
    #*======= STEP 1 : Add gradient infrastructure to tensor ========
    _original_init = Tensor.__init__ # store the original init to extend it

    def gradient_aware_init(self, data, requires_gradient=False):
        """Extend Tensor init to support gradinet tracking"""
        _original_init(self, data)
        self.requires_grad = requires_gradient
        self.grad = None
        
    Tensor.__init__ = gradient_aware_init

In [143]:
_original_init = Tensor.__init__

In [145]:
_original_init(self,data)

NameError: name 'self' is not defined